# Detect polarity in Amazon fine food reviews with TensorFlow
This work is part of a collection of practice sets called [NLP Starter](https://github.com/jamiemorales/project-nlp-starter).
It aims to help someone get started fast and gain a high-level understanding of the fundamental steps in the NLP lifecycle early on.
After completion, someone will have built intuition over the NLP lifecycle. 

## Step 0: Understand the problem
What we're trying to do here is to classify whether an Amazon review is positive or negative.

## Step 1: Set-up and understand data
In this step, we layout the tools we will need to solve the problem identified in the previous step. We want to inspect our data sources and explore the data itself to gain an understanding of the data for preprocessing and modeling.

In [30]:
# Set-up libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow import keras

In [55]:
# Load data
# Data source: https://www.kaggle.com/snap/amazon-fine-food-reviews
df = pd.read_csv('../00-Datasets/amazon-fine-food-reviews/Reviews.csv')
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [56]:
# Look at breakdown of label
df['Score'].value_counts()

5    363122
4     80655
1     52268
3     42640
2     29769
Name: Score, dtype: int64

## Step 2: Prepare data and understand some more
In this step, we perform the necessary transformations on the data so that the neural network would be able to understand it. Real-world datasets are complex and messy. For our purposes, most of the datasets we work on in this series require minimal preparation.

In [58]:
# Separate positive and negative reviews
df['Score'] = np.where(df['Score'] > 3, 1, 0)
df['Score'].value_counts()

0    568454
Name: Score, dtype: int64

In [67]:
# Split data into 80% training and 20% validation
sentences = df['Text']
labels = df['Score']

train_sentences, val_sentences, train_labels,val_labels = train_test_split(sentences, labels, test_size=0.2, random_state=0)

print(train_sentences.shape)
print(train_labels.shape)
print(val_sentences.shape)
print(val_labels.shape)

(454763,)
(454763,)
(113691,)
(113691,)


In [68]:
# Tokenize and pad
vocab_size = 10000
oov_token = '<00V>'
max_length = 500
padding_type = 'post'
trunc_type = 'post'

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

val_sequences = tokenizer.texts_to_sequences(val_sentences)
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

## Step 3: Build, train, and evaluate neural network
First, we design the neural network, e.g., sequence of layers and activation functions. 

Second, we train the neural network, we iteratively make a guess, calculate how accurate that guess is, and enhance our guess. The first guess is initialised with random values. The goodness or badness of the guess is measured with the loss function. The next guess is generated and enhanced by the optimizer function.

Lastly, we apply use the neural network on previously unseen data and evaluate the results.

In [74]:
# Build and train neural network
embedding_dim = 16
num_epochs = 3
batch_size = 100

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
             loss='binary_crossentropy',
              metrics=['accuracy']
             )

history = model.fit(train_padded, train_labels, batch_size=batch_size, epochs=num_epochs, verbose=2)

Train on 454763 samples
Epoch 1/3
454763/454763 - 33s - loss: 0.0149 - accuracy: 0.9988
Epoch 2/3
454763/454763 - 32s - loss: 3.6133e-06 - accuracy: 1.0000
Epoch 3/3
454763/454763 - 34s - loss: 2.5674e-07 - accuracy: 1.0000


Clearly that's overfitting but we will leave it here for now and address this in future examples.

## More

If you found this work interesting, you might like:

* Machine Learning Starter

* Deep Learning Starter

* Natural Language Processing Starter

You can find more at [github.com/jamiemorales](https://github.com/jamiemorales).

Datasets are not mine. List of sources: [datasets and sources]()

For sharing this work, here's how / the license: https://creativecommons.org/licenses/by-sa/4.0/